# For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


In [34]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\mabel\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0




geopy-1.21.0         | 58 KB     |            |   0% 
geopy-1.21.0         | 58 KB     | ##7        |  27% 
geopy-1.21.0         | 58 KB     | ########## | 100% 

geographiclib-1.50   | 34 KB     |            |   0% 
geographiclib-1.50   | 34 K

To create the  dataframe:
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into * one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [83]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned\n":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()
df.shape

(103, 3)

In [84]:
#Group all neighborhoods with the same postal code
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [85]:
print("Shape: ", df.shape)

Shape:  (103, 3)


#  To  get the latitude and the longitude coordinates of each neighborhood.

In [55]:
#Instalamos geocoder:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\mabel\anaconda3

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2




geocoder-1.38.1      | 53 KB     |            |   0% 
geocoder-1.38.1      | 53 KB     | ###        |  30% 
geocoder-1.38.1      | 53 KB     | ########## | 100% 

ratelim-0.1.6        | 6 KB      |            |   0% 
ratelim-0.1.6        | 6 K

Create a function to get the Lang Long from de Postal Code

In [82]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

Adding the Latitude and Longitude columns to the pandas dataframe

In [58]:

df['Latitude'], df['Longitude'] = zip(*df['PostalCode'].apply(get_geocoder))
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.785779,-79.157368
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


# Map of Toronto

In [60]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


With Folium Map

In [62]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df['Latitude'], df['Longitude'], df['PostalCode'], df['Borough'], df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

# Explore and cluster the neighborhoods in Toronto

In [68]:
# Printing all distinct Borough values
df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [69]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_all_df = df[df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print("Shape of All Toronto Data dataframe is - ",toronto_all_df.shape)
toronto_all_df.head(40)

Shape of All Toronto Data dataframe is -  (39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.678148,-79.295349
1,M4K,East Toronto,The Danforth West / Riverdale,43.683424,-79.354564
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668291,-79.315578
3,M4M,East Toronto,Studio District,43.648000,-79.339260
4,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415
5,M4P,Central Toronto,Davisville North,43.713171,-79.388870
6,M4R,Central Toronto,North Toronto West,43.714139,-79.406456
7,M4S,Central Toronto,Davisville,43.703327,-79.385649
8,M4T,Central Toronto,Moore Park / Summerhill East,43.690328,-79.383522
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686378,-79.402372


In [70]:
#Mapping only Toronto Data which was filtered earlier
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, post, borough, neigh in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['PostalCode'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

Declaring values API forsquare

In [71]:
CLIENT_ID = '2IVV030KWKTSMMIK0VSHUNFFFIZOVT2NGQ4ML4OB5DUHPL1P' #  Foursquare ID
CLIENT_SECRET = 'PN3ZEMKXWZOJCC4VSOYOYEPXVHHL5X0M0Y4OWDI1TWKEZYDM' #  Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2IVV030KWKTSMMIK0VSHUNFFFIZOVT2NGQ4ML4OB5DUHPL1P
CLIENT_SECRET:PN3ZEMKXWZOJCC4VSOYOYEPXVHHL5X0M0Y4OWDI1TWKEZYDM


Recommended places in each Borough in a 500 metres radius:

In [72]:
radius = 500
LIMIT = 1000

recommends = []
for lat, long, post, borough, neighborhood in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['PostalCode'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    postal_data = requests.get(url).json()["response"]['groups'][0]['items']
    for recommend_post in postal_data:
        recommends.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            recommend_post['venue']['name'], 
            recommend_post['venue']['location']['lat'], 
            recommend_post['venue']['location']['lng'],  
            recommend_post['venue']['categories'][0]['name']))

In [73]:
toronto_recommends = pd.DataFrame(recommends)
toronto_recommends.columns = ['PostalCode', 'Borough', 'Neighborhoods', 'Borough_Lat', 'Borough_Long', 'Venue_Name', 'Venue_Lat', 'Venue_Long', 'Venue_Category']
print("Shape of all the recommendations of all the Postal Codes in Toronto - ", toronto_recommends.shape)
toronto_recommends.head()

Shape of all the recommendations of all the Postal Codes in Toronto -  (1629, 9)


,PostalCode,Borough,Neighborhoods,Borough_Lat,Borough_Long,Venue_Name,Venue_Lat,Venue_Long,Venue_Category
0,M4E,East Toronto,The Beaches,43.678148,-79.295349,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.678148,-79.295349,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.678148,-79.295349,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.678148,-79.295349,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,Calvary Baptist Church,43.681059,-79.299246,Church


In [74]:
toronto_recommends.groupby(['PostalCode', 'Borough', 'Neighborhoods'])['Venue_Name'].count()

PostalCode  Borough           Neighborhoods                                                                                                   
M4E         East Toronto      The Beaches                                                                                                           5
M4K         East Toronto      The Danforth West / Riverdale                                                                                         5
M4L         East Toronto      India Bazaar / The Beaches West                                                                                      21
M4M         East Toronto      Studio District                                                                                                       3
M4N         Central Toronto   Lawrence Park                                                                                                         2
M4P         Central Toronto   Davisville North                                                             

In [75]:
print("Number of Unique Venue Categories are: ",len(toronto_recommends['Venue_Category'].unique()))
toronto_recommends['Venue_Category'].unique()

Number of Unique Venue Categories are:  221


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Church',
       'Business Service', 'Discount Store', 'Park', 'Bus Line',
       'Grocery Store', 'Gym', 'Fast Food Restaurant', 'Ice Cream Shop',
       'Fish & Chips Shop', 'Sushi Restaurant', 'Liquor Store',
       'Italian Restaurant', 'Pet Store', 'Steakhouse', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Restaurant', 'Board Shop',
       'Pizza Place', 'Food & Drink Shop', 'Government Building',
       'Baseball Field', 'Night Market', 'Swim School', 'Breakfast Spot',
       'Department Store', 'Gym Pool', 'Playground', 'Garden',
       'Dessert Shop', 'Café', 'Indian Restaurant', 'Thai Restaurant',
       'Seafood Restaurant', 'Diner', 'Coffee Shop', 'Toy / Game Store',
       'Gas Station', 'Farmers Market', 'Convenience Store',
       'Supermarket', 'Light Rail Station', 'Skating Rink', 'Candy Store',
       'Japanese Restaurant', 'Bakery', 'Jewelry Store', 'Gastropub',
       'Butcher', 'General Enter

Which are the venues for each area:

In [76]:
# getting all the values as columns for all areas
toronto_recommends = toronto_recommends.drop(['Borough_Lat','Borough_Long','Venue_Lat','Venue_Long'],axis=1)
toronto_recommends_df = pd.get_dummies(toronto_recommends, columns=['Venue_Category'],prefix = "", prefix_sep = "")
toronto_recommends_df.head(40)

,PostalCode,Borough,Neighborhoods,Venue_Name,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,Glen Manor Ravine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,The Big Carrot Natural Food Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0

In [77]:
toronto_venues_freq = toronto_recommends_df.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()
toronto_venues_freq.head(40)

,PostalCode,Borough,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000

Top ten values

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# Create a new dataframe
all_toronto_venues = pd.DataFrame(columns=columns)
all_toronto_venues['PostalCode'] = toronto_venues_freq['PostalCode']
all_toronto_venues['Borough'] = toronto_venues_freq['Borough']
all_toronto_venues['Neighborhoods'] = toronto_venues_freq['Neighborhoods']
for ind in np.arange(toronto_venues_freq.shape[0]):
    row_categories = toronto_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    all_toronto_venues.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

all_toronto_venues.sort_values(freqColumns, inplace=True)
all_toronto_venues

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,M6J,West Toronto,Little Portugal / Trinity,Bar,Restaurant,Cocktail Bar,Coffee Shop,Wine Bar,Vietnamese Restaurant,Asian Restaurant,Yoga Studio,New American Restaurant,Seafood Restaurant
3,M4M,East Toronto,Studio District,Baseball Field,Government Building,Night Market,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,M4K,East Toronto,The Danforth West / Riverdale,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop
4,M4N,Central Toronto,Lawrence Park,Bus Line,Swim School,Yoga Studio,Food Court,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
24,M5S,Downtown Toronto,University of Toronto / Harbord,Café,Bookstore,Italian Restaurant,Park,Restaurant,Japanese Restaurant,Comfort Food Restaurant,Chinese Restaurant,French Restaurant,Bubble Tea Shop
15,M5C,Downtown Toronto,St. James Town,Café,Coffee Shop,Cocktail Bar,Restaurant,American Restaurant,Cosmetics Shop,Hotel,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
35,M6S,West Toronto,Runnymede / Swansea,Café,Coffee Shop,Italian Restaurant,Gastropub,Electronics Store,Pub,Sushi Restaurant,Pizza Place,Restaurant,Falafel Restaurant
29,M6G,Downtown Toronto,Christie,Café,Grocery Store,Coffee Shop,Playground,Park,Athletics & Sports,Baby Store,Candy Store,Farmers Market,Ethiopian Restaurant
32,M6K,West Toronto,Brockton / Parkdale Village / Exhibition Place,Coffee Shop,Café,Diner,Gift Shop,Thrift / Vintage Store,Restaurant,Accessories Store,Lounge,Italian Restaurant,Japanese Restaurant
26,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,Coffee Shop,Café,Restaurant,French Restaurant,Park,Lounge,Speakeasy,Bar,Italian Restaurant,Gym / Fitness Center


Clustering areas wiht k-means algorithm 

We will establish 5 cluster

In [80]:
# set number of clusters
kclusters = 5

toronto_venues_freq_clustering = toronto_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues_freq_clustering)

#No data for a particular postal code so removed it
toronto_clustered_df = toronto_all_df.drop(toronto_all_df.index[toronto_all_df.PostalCode=='M5N'])
toronto_clustered_df['Cluster'] = kmeans.labels_

toronto_clustered_df = toronto_clustered_df.join(all_toronto_venues.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_clustered_df #Df with cluster labels

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4K,East Toronto,The Danforth West / Riverdale,43.683424,-79.354564,0,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop
30,M6G,Downtown Toronto,Christie,43.668602,-79.420387,0,Café,Grocery Store,Coffee Shop,Playground,Park,Athletics & Sports,Baby Store,Candy Store,Farmers Market,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,43.713171,-79.388870,0,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop
31,M6H,West Toronto,Dufferin / Dovercourt Village,43.664910,-79.438664,0,Grocery Store,Park,Bus Line,Art Gallery,Athletics & Sports,Gym / Fitness Center,Gym,Café,Bakery,Bank
6,M4R,Central Toronto,North Toronto West,43.714139,-79.406456,0,Playground,Gym Pool,Park,Garden,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
10,M4W,Downtown Toronto,Rosedale,43.681893,-79.376706,0,Playground,Park,Grocery Store,Candy Store,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
34,M6P,West Toronto,High Park / The Junction South,43.658720,-79.463150,0,Sandwich Place,Convenience Store,Metro Station,Gym / Fitness Center,Park,Residential Building (Apartment / Condo),Gas Station,Yoga Studio,Fast Food Restaurant,Farmers Market
32,M6J,West Toronto,Little Portugal / Trinity,43.648636,-79.417752,1,Bar,Restaurant,Cocktail Bar,Coffee Shop,Wine Bar,Vietnamese Restaurant,Asian Restaurant,Yoga Studio,New American Restaurant,Seafood Restaurant
3,M4M,East Toronto,Studio District,43.648000,-79.339260,1,Baseball Field,Government Building,Night Market,Dumpling Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
25,M5S,Downtown Toronto,University of Toronto / Harbord,43.663281,-79.398088,1,Café,Bookstore,Italian Restaurant,Park,Restaurant,Japanese Restaurant,Comfort Food Restaurant,Chinese Restaurant,French Restaurant,Bubble Tea Shop


Mapping cluster

In [81]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_clustered_df['Latitude'], toronto_clustered_df['Longitude'], toronto_clustered_df['PostalCode'], toronto_clustered_df['Borough'], toronto_clustered_df['Neighborhood'], toronto_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters